In [1]:
import os
import sys
import pickle

import cv2
import numpy as np
import neuroglancer
from tqdm import tqdm
from cloudvolume import CloudVolume

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.sqlcontroller import SqlController
from utils import get_structure_number, get_segment_properties, NumpyToNeuroglancer

Connecting dklab@192.168.1.12:3306


In [3]:
animal = 'MD589'
downsample_factor = 16

OUTPUT_DIR_PATH = os.path.join('./')
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full'
NG_DIR_PATH = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/structures')
# NG_DIR_PATH = os.path.join('./')

sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

In [4]:
def draw_numpy(section_structure_polygons, section_start, section_end, draw_lines=True):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    color = get_structure_number(structure)
                    
                    if draw_lines:
                        cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    else:
                        for point in polygon:
                            cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

def get_ranges(start, end, interval):
    ranges = []
    
    cur = start
    while cur + interval <= end:
        next = cur + interval
        ranges.append((cur, next))
        cur = next
    ranges.append((cur, end))
    return ranges

In [5]:
draw_lines = True
polygons_path = os.path.join(OUTPUT_DIR_PATH, f'MD589_CVAT_downsample{downsample_factor}.pkl')

with open(polygons_path, 'rb') as file:
    section_structure_polygons = pickle.load(file)
volume = draw_numpy(section_structure_polygons, 0, num_section, draw_lines=draw_lines)

100%|██████████| 447/447 [00:47<00:00,  9.34it/s]


In [7]:
precompute_path = os.path.join(NG_DIR_PATH, f'{animal}_down{downsample_factor}_annotations')

ng = NumpyToNeuroglancer(volume, scales)
ng.preview()
ng.init_precomputed(precompute_path)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[ 7232.  7232. 20000.] is added to:
http://127.0.0.1:33733/v/959b400c62f7a8250a17143c0da01fa5dbcac368/


Tasks:   0%|          | 0/14 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[2731, 2025, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[2731, 2025, 447], dtype=int32)



Downloading:  33%|███▎      | 342/1024 [00:02<00:29, 23.37it/s]


Downloading: 100%|██████████| 352/352 [00:04<00:00, 77.75it/s] 

Downloading: 100%|██████████| 352/352 [00:04<00:00, 76.70it/s] 

Downloading:  47%|████▋     | 483/1024 [00:04<04:04,  2.21it/s]


...

Uploading: 100%|██████████| 256/256 [00:03<00:00, 73.72it/s]

Uploading: 100%|██████████| 256/256 [00:03<00:00, 73.75it/s]

Uploading: 100%|██████████| 64/64 [00:01<00:00, 57.42it/s] ]

Uploading: 100%|██████████| 256/256 [00:04<00:00, 62.75it/s]

Uploading: 100%|██████████| 256/256 [00:04<00:00, 56.50it/s]

Uploading: 100%|██████████| 16/16 [00:00<00:00, 59.18it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 104.32it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 118.75it/s]

Uploading: 100%|██████████| 16/16 [00:00<00:00, 18.65it/s]]

Uploading: 100%|██████████| 4/4 [00:00<00:00,  6.20it/s]

Uploading: 100%|██████████| 64/64 [00:00<00:00, 96.31it/s]

Uploading: 100%|██████████| 16/16 [00:00<00:00, 138.42it/s]

Uploading: 100%|██████████| 4/4 [00:00<00:00, 33.02it/s]

Uploading: 100%|██████████| 16/16 [00:00<00:00, 104.04it/s]

Uploading: 100%|██████████| 4/4 [00:00<00:00, 82.30it/s]

Downloading:  58%|█████▊    | 262/448 [00:00<00:00, 700.70it/s]


Downloading:  62%|██████▎   | 280/448 [00:01<00:03, 46.89it/s]

Downloading:  19%|█▉        | 85/448 [00:03<00:00, 738.40it/s]


Downloading: 100%|██████████| 56/56 [00:05<00:00, 10.97it/s]/s]

Downloading:  26%|██▌       | 115/448 [00:05<00:00, 721.24it/s]


Downloading: 100%|██████████| 56/56 [00:06<00:00,  8.01it/s]s]

Downloading: 100%|██████████| 448/448 [00:08<00:00, 50.74it/s] 

Downloading: 100%|██████████| 56/56 [00:08<00:00,  6.30it/s]


Tasks: 100%|██████████| 999/999 [00:00<00:00, 1544.64it/s]
